In [ ]:
import pandas as pd
import os

# Function to process each sheet
def process_sheet(sheet):
    # Find the header row
    header_row = None
    for index, row in sheet.iterrows():
        if ('Entity' in row.values) and ('RTN Level 4' in row.values) and ('Account' in row.values):
            header_row = index
            break
    
    if header_row is None:
        print("Header not found in the sheet.")
        return

    # Row above header
    year_row = header_row - 1

    # Identify the Mxx, FY, and YTDxx columns and concatenate them with the row above
    for col in sheet.columns:
        if re.match(r'M\d{2}', col) or col.startswith('FY') or re.match(r'YTD\d{2}', col):
            # Concatenate current cell with the one above
            for idx in range(header_row + 1, len(sheet)):
                sheet.at[idx, col] = f"{sheet.at[year_row, col]} {sheet.at[idx, col]}"

    # Display for testing
    print(f"Processed sheet from header row {header_row}.")

# Function to create or overwrite the workbook
def create_or_overwrite_workbook(file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File '{file_path}' already exists. It has been deleted and will be recreated.")
    else:
        print(f"File '{file_path}' does not exist. It will be created.")

    # Create an empty workbook to write processed sheets to
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        # An empty workbook is created at this point
        pass

# Main processing function
def main():
    # Get current year and month from user
    current_year = input("Enter current year (e.g., 2024): ")
    current_month = input("Enter current month (1-12): ")

    # Specify the file path where the processed file will be saved
    processed_file = 'processed_file.xlsx'

    # Create or overwrite the workbook
    create_or_overwrite_workbook(processed_file)

    # Load the workbook
    excel_file = 'your_file.xlsx'  # Replace with your input file path
    xls = pd.ExcelFile(excel_file)
    
    # Process each sheet
    for sheet_name in xls.sheet_names:
        if 'excl' not in sheet_name:
            sheet = pd.read_excel(xls, sheet_name=sheet_name)
            process_sheet(sheet)

            # Save the processed sheet to the new workbook
            with pd.ExcelWriter(processed_file, engine='openpyxl', mode='a') as writer:
                sheet.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Processing complete. Data saved to '{processed_file}'.")

if __name__ == "__main__":
    main()
